# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

I Observed that creating an exploratory-interactive geospatial, Weather information, heat maps, and visualization through Google API, and other similar API applications play a remarkable role to exploit the potential socio-economic value buried inside monotonously large datasets.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import collections
collections.Iterable = collections.abc.Iterable

# Import API key
from api_keys import g_key, maps_javascript_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
Cities_DF = pd.read_csv('../output_data/Cities.csv')
Cities_DF.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key:
gmaps.configure(api_key=maps_javascript_api_key)

In [5]:
#get locations and humidity
city_locations = Cities_DF[['Lat', 'Lng']].astype(float)
#print(city_locations)
humidity = Cities_DF['Humidity'].astype(float)

# Create a humidity heat map layer:


heat_layer = gmaps.heatmap_layer(city_locations,
                                 weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig=gmaps.figure()
fig.add_layer(heat_layer)

#plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create new data frame with requested weather conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows with null values

filtered_weather_df=Cities_DF[(Cities_DF['Max Temp'] < 80) &
                             (Cities_DF['Max Temp'] > 70) &
                             (Cities_DF['Wind Speed'] < 10) &
                             (Cities_DF['Cloudiness'] == 0)]

filtered_weather_df=filtered_weather_df.dropna() # drop any rows will null values

filtered_weather_df



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = filtered_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : maps_javascript_api_key}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print('-------------------------------')

Retrieving Results for Index 70: ponta do sol.
Closest hotel is Pousada Suítes do Lago.
-------------------------------
Retrieving Results for Index 88: cidreira.
Closest hotel is Hotel Castelo.
-------------------------------
Retrieving Results for Index 138: morondava.
Closest hotel is Palissandre Cote Ouest resort & SPA.
-------------------------------
Retrieving Results for Index 139: vaini.
Closest hotel is Dandeli Aura Jungle Stay.
-------------------------------
Retrieving Results for Index 176: nador.
Closest hotel is Hôtel Mercure Rif Nador.
-------------------------------
Retrieving Results for Index 297: mogok.
Closest hotel is MOGOK HILL HOTEL.
-------------------------------
Retrieving Results for Index 444: birjand.
Missing field/result... skipping.
-------------------------------


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
fig=gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_locations,
                                 weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map

fig.add_layer(heat_layer)

fig.add_layer(markers)
plt.savefig("vacationheatmapwithhotels.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>